In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [2]:
import json

with open('/home/husein/news/populate-news.json') as fopen:
    data = json.load(fopen)

In [3]:
data.keys()

dict_keys(['text', 'title'])

In [4]:
from tqdm import tqdm
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [5]:
len(data['title'][0].split())

12

In [6]:
with tf.io.gfile.GFile('newstitle.tsv', "w") as outfile:
    for i in range(len(data['title'])):
        c = cleaning(data['text'][i])
        if len(data['title'][i].split()) > 3 and len(c) > 20 and 'javascript' not in c and 'register in order to view' not in c:
            outfile.write("%s\t%s\n" % (c, cleaning(data['title'][i])))

In [7]:
def news_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'newstitle.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def news_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['tajuk: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [8]:
t5.data.TaskRegistry.remove('news_dataset')
t5.data.TaskRegistry.add(
    'news_dataset',
    dataset_fn = news_dataset,
    splits = ['train'],
    text_preprocessor = [news_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


In [9]:
nq_task = t5.data.TaskRegistry.get("news_dataset")
ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [10]:
next(r)

{'inputs_plaintext': b'tajuk: Ada Eks Orang Istana Jadi Direksi Jiwasraya, Ini Faktanya - Komisi VI DPR RI meminta anggota dewan direksi PT Asuransi Jiwasraya (Persero) periode 2013-2018 dicekal. Mereka dianggap yang paling bertanggung jawab atas kusutnya permasalahan pembayaran polis bancassurance nasabah.Mereka dinilai lalai dalam menjaga prinsip kehati-hatian saat menjual produk JS Saving Plan dalam kurun waktu 2014-2018 hingga mengakibatkan kerugian bagi perusahaan dan nasabah.Salah satu direksi Jiwasraya saat itu adalah Hari Prasetyo yang menjabat sebagai direktur keuangan. Hari juga ditunjuk Kepala Staf Kepresidenan Jenderal TNI (Purn) Moeldoko sebagai Tenaga Ahli Utama Kedeputian III Bidang Kajian dan Pengelolaan Isu-isu Ekonomi Strategis.Dia bergabung berbarengan dengan Ali Mochtar Ngabalin yang ditunjuk sebagai Tenaga Ahli Utama Kedeputian IV Bidang Komunikasi Politik dan Diseminasi Informasi.Namun, untuk periode sekarang, belum diketahui apakah Hari masih menjabat sebagai ten